In [1]:
import os 
import json
import tqdm as tqdm
import time
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


download the dataset
this images.rar is used for another reason


!tar -xzvf "/content/drive/MyDrive/figure_extraction/train-0.tar.gz" -C "/content/drive/MyDrive/figure_extraction/"


Extract the tar dataset
this is used for other reason this images.rar

extract the labeled dataset


this for file movement


Load the jason labels

In [4]:
#os.listdir("../Trash")
#os.chdir("drive/My Drive")
os.getcwd()

!ls 
os.chdir("/content/drive/My Drive/images")

drive  sample_data


In [5]:

with open ("publaynet/train.json","r") as train_file:
     train_dict = json.load(train_file)
     pass

get image

In [6]:
images = dict()


for image in os.listdir("publaynet/train/"):
    images[image] = 1


train_images = list()
for image in train_dict["images"]:
  try:
    if images[image["file_name"]]:train_images.append(image)
  except:pass
train_dict["images"] = train_images

del train_images


In [7]:
print(train_dict["images"][1])

{'file_name': 'PMC3866684_00012.jpg', 'height': 811, 'id': 8, 'width': 613}


load images in dict for fast accessing

In [8]:
dict_image = dict()
for image in train_dict["images"]:
    dict_image[str(image["id"])] = image["file_name"]
print(len(dict_image))

39158


In [9]:
len(train_dict["annotations"])

3263046

In [10]:
train_dict["annotations"][0]

{'segmentation': [[52.38,
   444.87,
   291.97,
   444.87,
   291.97,
   456.42,
   291.97,
   456.42,
   291.97,
   465.8,
   291.97,
   465.8,
   291.97,
   476.27,
   291.97,
   476.27,
   291.97,
   487.8,
   291.97,
   487.8,
   291.97,
   498.26,
   86.41,
   498.26,
   86.41,
   508.73,
   40.42,
   508.73,
   40.42,
   497.17,
   40.42,
   497.17,
   40.42,
   486.71,
   40.42,
   486.71,
   40.42,
   476.27,
   40.42,
   476.27,
   40.42,
   466.88,
   40.42,
   466.88,
   40.42,
   455.33,
   52.38,
   455.33,
   52.38,
   444.87]],
 'area': 13787.433082525036,
 'iscrowd': 0,
 'image_id': 0,
 'bbox': [40.42, 444.87, 251.55, 63.86],
 'category_id': 1,
 'id': 0}

In [11]:
for anno in train_dict["annotations"]:
  del anno["segmentation"]
  del anno["area"]
  del anno["iscrowd"]

remove annotations thats not belong to train images

In [12]:
images_anno = list()
print(len(train_dict["annotations"]))
for anno in train_dict["annotations"]:
  try:
    if dict_image[str(anno["image_id"])]: images_anno.append(anno)
  except:pass
print(len(images_anno))
train_dict["annotations"] = images_anno
del images_anno
#train_dict["images"] = images_train

3263046
381317


In [13]:
with open("publaynet/train_2.json","w") as file:
  json.dump(train_dict,file)

In [31]:
with open ("publaynet/train_2.json","r") as train_file:
     train_dict = json.load(train_file)
     pass

In [32]:
train_dict["images"][0]

{'file_name': 'PMC3866684_00001.jpg', 'height': 811, 'id': 1, 'width': 613}

In [33]:
print(len(train_dict["annotations"]))

381317


In [34]:
text = 0
title = 0
List = 0
table = 0
figure = 0
for anno in train_dict["annotations"]:
     if anno["category_id"] == 5 :figure += 1
     elif anno["category_id"] == 1 :text += 1
     elif anno["category_id"] == 2 :title += 1
     elif anno["category_id"] == 3 :List += 1
     else : table += 1
print(figure)
print(text)
print(List)
print(table)
print(title)   

12628
273615
9815
11993
73266


## Sampling the data
as we can see above the annotations are biased to the not_figure boxes, and that's normal because the content of reasearch paper are almost text data.
since we are having a binary classification we don't need a big amount of data so we can remove this bias by sampling not_figure boxes. to be almost the same size with the number of figure boxes.

we are going to use this distribution:



1.   12000 figure
2.   13000 not_figure ditributed as follow :

      *   5000 texts
      *   4000 tables
      *   2000 lists
      *   2000 titles









In [35]:
import random

annotation_text = list()
annotation_title = list()
annotation_list = list()
annotation_table = list()
annotation_figure = list()

for anno in train_dict["annotations"]:
  if anno["category_id"] == 1:annotation_text.append(anno["id"])
  elif anno["category_id"] == 2 :annotation_title.append(anno["id"])
  elif anno["category_id"] == 3 :annotation_list.append(anno["id"])
  elif anno["category_id"] == 4 :annotation_table.append(anno["id"])
  else: annotation_figure.append(anno["id"])

annotation_text = random.sample(annotation_text,5000)
annotation_title = random.sample(annotation_title,2000)
annotation_list = random.sample(annotation_list,2000)
annotation_table = random.sample(annotation_table,4000)
annotation_figure = random.sample(annotation_figure,12000)



remove all unnecessary annotation and images ( that doesn't belong to the selected 25000 )

In [36]:
# list of all annoation id
annotation = annotation_text + annotation_title + annotation_list + annotation_table + annotation_figure
annotation = random.sample(annotation,len(annotation))

# create a dict for the list of annotations to fast accessing
annotation_dict = dict()

for anno in annotation:
  annotation_dict[anno] = 1

# this is a dict of the images to keep (who have annotation)
images_to_keep = dict()

# list contain finale annotations to assign it to the final dict
final_annotations = list()

for anno in train_dict["annotations"]:

  try:
     if annotation_dict[anno["id"]]: 
       final_annotations.append(anno)
       images_to_keep[anno["image_id"]] = 1
  except:pass

train_dict["annotations"] = final_annotations

# list contain finale images to assign it to the final dict
final_images= list()
for image in train_dict["images"]:
  try:
     if images_to_keep[image["id"]]: final_images.append(image)
  except:pass


train_dict["images"] = final_images

del final_images, final_annotations, annotation, annotation_dict

### change the label of annotations by transforming all not figure images to 0, and the figure to 1

In [37]:
for annotation in train_dict["annotations"]:
  annotation["category_id"] = 1 if annotation["category_id"] == 5 else 0



### create a dictionary have as key the image name and as value a dictionary have two items the first is a list of annotations category that's belong to the image and the seconde is the coordinates of annotations
to avoid opening images many times (the case where the image have more than one annotation) or pass through all annotation for each images

In [9]:
# this dictionary is used as reference of the image which have as key the image_id and as value the image name
images_name = dict()

images_dict = dict()

for image in train_dict["images"]:
  images_dict[image["file_name"]] = {
                   "annotations_category": [],
                   "annotations_bbox": [] 
                  }
  images_name[image["id"]] = image["file_name"]



for annotation in train_dict["annotations"]:
  images_dict[images_name[annotation["image_id"]]]["annotations_category"].append(annotation["category_id"])
  images_dict[images_name[annotation["image_id"]]]["annotations_bbox"].append(annotation["bbox"])

i = 0
for key, value  in images_dict.items():
  i += len(value["annotations_category"])

train_dict = images_dict

In [11]:
with open("publaynet/train_3.json","w") as file:
  json.dump(train_dict,file)

In [ ]:
import shutil
path = "drive/MyDrive/fig/dataset_math/not_figure"
not_fig = os.listdir(path)
"""
for what in not_fig:
    old = os.path.join(path,what)
    new_name = '_'+what
    new_name = os.path.join(path,new_name)
    os.rename(old,new_name)
"""
for what in not_fig:
    image = os.path.join(path,what)

    shutil.copy(image, "drive/MyDrive/fig/dataset_12000/not_figure") 

In [ ]:
print(len(os.listdir("/content/drive/MyDrive/fig/new_data")))
os.getcwd()
!wget -c "https://download936.mediafire.com/ny0u64p535pg/ovc8y72nozvtcwh/images.rar" --no-check-certificate



None
--2021-09-07 21:25:43--  https://download936.mediafire.com/ny0u64p535pg/ovc8y72nozvtcwh/images.rar
Resolving download936.mediafire.com (download936.mediafire.com)... 205.196.121.131
Connecting to download936.mediafire.com (download936.mediafire.com)|205.196.121.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 666068584 (635M) [application/x-rar]
Saving to: ‘images.rar’

images.rar          100%[===================>] 635.21M  26.0MB/s    in 25s     

2021-09-07 21:26:09 (25.3 MB/s) - ‘images.rar’ saved [666068584/666068584]



In [ ]:
!unrar x "/content/drive/MyDrive/fig/images.rar" "/content/drive/MyDrive/fig/images/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/fig/images.rar

Creating    /content/drive/MyDrive/fig/images/images                  OK
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_10_page1.jpg       0%  OK 
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_10_page2.jpg       0%  OK 
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_10_page3.jpg       0%  OK 
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_10_page4.jpg       0%  OK 
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_10_page5.jpg       0%  OK 
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_10_page6.jpg       0%  OK 
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_10_page7.jpg       0%  OK 
Extracting  /content/drive/MyDrive/fig/images/images/aaai10_1_page1.jpg       0%  OK 
Extracting  /content/drive/MyDrive/f